In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
property_data = pd.read_csv('../../properati-AR-2017-08-01-properties-sell.csv')

# ANALISIS DE LAS DESCRIPCIONES

In [3]:
#Primero vamos a ver cuantas propiedades tienen descripciones y cuantas titulo.
property_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 27 columns):
id                            187482 non-null object
created_on                    187482 non-null object
operation                     187482 non-null object
property_type                 187482 non-null object
place_name                    187447 non-null object
place_with_parent_names       187482 non-null object
country_name                  187482 non-null object
state_name                    187482 non-null object
geonames_id                   158318 non-null float64
lat-lon                       118761 non-null object
lat                           118761 non-null float64
lon                           118761 non-null float64
price                         154443 non-null float64
currency                      154441 non-null object
price_aprox_local_currency    154443 non-null float64
price_aprox_usd               154443 non-null float64
surface_total_in_m2           12502

## Analisis de los titulos.

In [91]:
def parseoTitulo(x):
    
    tipo = ""
    tiposViviendas = ['casa', 'ph','p.h.', 'departamento','depto','dpto','dto',
                      'chalet','duplex','duples','triplex','edificio','semipiso','piso',
                      'p.baja','planta baja', 'p.b.','casona', 'loft', 'cabaña', 'mansion',
                      'local','quinta','bungalow','oficina',
                      'terreno','lote']
    x=x.lower()
    
    for i in tiposViviendas:
        aux = ""
        
        if( x.find(i) >= 0 ):
            aux += i
        if( aux != "" and aux not in tipo):
            tipo+=aux
            tipo+=" "

    if(tipo !=""): return tipo
    return x

def parseoCaracteristica(x):
    
    caract = []
    caractViviendas = ['estrenar','nueva', 
                       'monoambiente','ambiente','amb', 
                       'dormitorio','dorm','hab','habitaciones',
                       'baño','estac', 'estacionamiento','cochera','balcon',"pileta"]
    x=x.lower()
    
    for j in caractViviendas:
        aux = ""
        indice = x.find(j)
        if( indice >=0 ):
            
            if( j not in caract ):
                "Si son ambientes, vemos cuantos"
                if(j == 'ambiente' or j=='amb'):
                    aux = x[indice - 2]
                    aux += " "
                
                aux += j
                caract.append( aux )
                
    if(len(caract) != 0): return caract
    return x


def separarCaracteristicas(x, indice):
        return x[indice]

In [92]:
titulos = property_data[['title','price_per_m2','id']]

#Vamos a intentar parsear por tipo de casa
#Primero me quedo con los titulos antes de aplicar ninguna operacion
caracteristicas = titulos[['title', 'id']]

#Parseo el titulo y me quedo con el tipo de vivienda.
tiposVivienda = titulos[['title', 'id', 'price_per_m2']]
tiposVivienda['title'] = tiposVivienda['title'].apply(lambda x: parseoTipo(x))

#Parseo el titulo y me quedo con las caracteristicas.
caracteristicas['title'] = caracteristicas['title'].apply(lambda x: parseoCaracteristica(x))

#Mergeo los dos separados
completo = pd.merge(caracteristicas, tiposVivienda, on='id', how='inner')
completo.rename(columns={'title_x':'caracteristicas'}, inplace=True)
completo.rename(columns={'title_y':'tipo'}, inplace=True)

completo.head(10)

"""
HAY MUY POCOS QUE TIENEN CARACTERISTICAS. HAY QUE VER SI CON LA DESCRIPCION PODEMOS EXPANDIR
UN POCO MAS LA INFO. ES MUY POCA.
"""


/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,caracteristicas,id,tipo,price_per_m2
0,casa en venta,c8a66ee9e3da083ab66dacd165e8a5cc718b1cf2,casa,1290.322581
1,casa en venta,a205c53378e7879cf7519659317fb410c060d20e,casa,1000.000000
2,[2 amb],e06d6f864a0b91346594da4d27ceddcef0947ea6,dto,NaN
3,chalet en venta,a07927207d2a03de5f4ba7fd76f670d6285a3315,chalet,NaN
4,casa en venta,9446052c949050359fb08b9f1ba83ba2eaa20c99,casa,1796.116505
5,departamento en venta ubicado en arcos de pilar,4d3b492ed5f9c74cdf33b56b98328e5c950e3f9c,departamento,1796.296296
6,local en venta ubicado en san isidro,535441f569473f6e5d687c38b27d9b9c6e69e284,local,NaN
7,"[2 ambiente, 2 amb]",7188b6176d94ef327da690749469b6065acd7426,departamento,NaN
8,casa en venta ubicado en quilquihue,27d87ac71e7c00afa9fa7428b943ec263ff68bb4,casa,1846.153846
9,casa en venta ubicado en rio hermoso,df77a06585671e637d2d86f8d22b70d61cfecb18,casa,1875.000000


In [ ]:

#Agrupo por tipo de vivienda
#ordenado = completo.groupby(['tipo','id','price_per_m2']).count().reset_index()


#completo['tipo','id'].groupby(['tipo']).agg(np.size).head(10)
#grouped = grouped.agg('caracteristicas')
#grouped.head(10)
#grouped = grouped.agg(np.size)
#grouped.sort_values('caracteristicas', ascending =False).head(10)